In [2]:
import streamlit as st
import requests
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

%load_ext streamlit

def get_stock_news(page_number, start_date=None, end_date=None):
    date_filter = f"&sd={start_date.strftime('%Y%m%d')}" if start_date else ""
    date_filter += f"&ed={end_date.strftime('%Y%m%d')}" if end_date else ""

    headlines = []
    linkz = []
    resp = requests.get(f"https://economictimes.indiatimes.com/markets/stocks/rssfeeds/2146842.cms?page={page_number}{date_filter}")
    soup = BeautifulSoup(resp.content, features='xml')
    k = soup.findAll('title')
    lnk = soup.findAll('link')

    for txt in k:
        headlines.append(txt.get_text())

    for links in lnk:
        linkz.append(links.get_text())

    linkz = linkz[2:len(linkz)]
    headlines = headlines[2:len(headlines)]

    return list(zip(headlines, linkz))

def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = analyzer.polarity_scores(text)['compound']
    return sentiment_score

def main():
    st.title("Stock News Headlines")


    search_query = st.sidebar.text_input("Search News", "")
    start_date = st.sidebar.date_input("Start Date", (datetime.now() - timedelta(days=7)).date())
    end_date = st.sidebar.date_input("End Date", datetime.now().date())


    news_list = get_stock_news(1, start_date, end_date)


    filtered_news = [(headline, link) for headline, link in news_list
                     if search_query and search_query.lower() in headline.lower()]


    for i, (headline, link) in enumerate(filtered_news):
        st.write(f"{headline} - {link}")
        st.markdown("---")

The streamlit module is not an IPython extension.
